In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from IPython.display import display
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
from joblib import dump

In [2]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')
df = df.drop(columns=['id'])
df['bmi'] = df['bmi'].fillna(df['bmi'].mean())


In [3]:
cats = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
encoder = OrdinalEncoder()
df[cats] = encoder.fit_transform(df[cats])
df = pd.get_dummies(df, columns = ["work_type", "Residence_type"], dtype=int)
df

,gender,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,smoking_status,stroke,work_type_0.0,work_type_1.0,work_type_2.0,work_type_3.0,work_type_4.0,Residence_type_0.0,Residence_type_1.0
0,1.0,67.0,0,1,1.0,228.69,36.600000,1.0,1,0,0,1,0,0,0,1
1,0.0,61.0,0,0,1.0,202.21,28.893237,2.0,1,0,0,0,1,0,1,0
2,1.0,80.0,0,1,1.0,105.92,32.500000,2.0,1,0,0,1,0,0,1,0
3,0.0,49.0,0,0,1.0,171.23,34.400000,3.0,1,0,0,1,0,0,0,1
4,0.0,79.0,1,0,1.0,174.12,24.000000,2.0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,0.0,80.0,1,0,1.0,83.75,28.893237,2.0,0,0,0,1,0,0,0,1
5106,0.0,81.0,0,0,1.0,125.20,40.000000,2.0,0,0,0,0,1,0,0,1
5107,0.0,35.0,0,0,1.0,82.99,30.600000,2.0,0,0,0,0,1,0,1,0
5108,1.0,51.0,0,0,1.0,166.29,25.600000,1.0,0,0,0,1,0,0,1,0


In [5]:
num = ["age", "avg_glucose_level", "bmi"]
scaler = StandardScaler()
df[num] = scaler.fit_transform(df[num])
df

,gender,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,smoking_status,stroke,work_type_0.0,work_type_1.0,work_type_2.0,work_type_3.0,work_type_4.0,Residence_type_0.0,Residence_type_1.0
0,1.0,1.051434,0,1,1.0,2.706375,1.001234e+00,1.0,1,0,0,1,0,0,0,1
1,0.0,0.786070,0,0,1.0,2.121559,4.393074e-16,2.0,1,0,0,0,1,0,1,0
2,1.0,1.626390,0,1,1.0,-0.005028,4.685773e-01,2.0,1,0,0,1,0,0,1,0
3,0.0,0.255342,0,0,1.0,1.437358,7.154182e-01,3.0,1,0,0,1,0,0,0,1
4,0.0,1.582163,1,0,1.0,1.501184,-6.357112e-01,2.0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,0.0,1.626390,1,0,1.0,-0.494658,4.393074e-16,2.0,0,0,0,1,0,0,0,1
5106,0.0,1.670617,0,0,1.0,0.420775,1.442949e+00,2.0,0,0,0,0,1,0,0,1
5107,0.0,-0.363842,0,0,1.0,-0.511443,2.217363e-01,2.0,0,0,0,0,1,0,1,0
5108,1.0,0.343796,0,0,1.0,1.328257,-4.278451e-01,1.0,0,0,0,1,0,0,1,0


In [7]:
X, y = df.drop('stroke', axis=1), df['stroke']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
smote = SMOTE(random_state=42)
x_smote, y_smote = smote.fit_resample(X_train, y_train)

rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [300, 500, 1000],
    'max_depth': [5, 10, None],
}
grid_search = GridSearchCV(rf, param_grid)
grid_search.fit(x_smote, y_smote)

best_rdf = grid_search.best_estimator_
y_pred = best_rdf.predict(X_test)
y_prob = best_rdf.predict_proba(X_test)[:, 1]

In [9]:
dump(best_rdf, "./utils/model.joblib")
dump(scaler, "./utils/scaler.joblib")
dump(encoder, "./utils/encoder.joblib")

['./utils/encoder.joblib']